<a href="https://colab.research.google.com/github/ozgekokyay/tensorflow-deep-learning/blob/main/tf_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras import layers

In [ ]:
def relu_bn(x):
  relu = layers.ReLU()(x)
  bn = layers.BatchNormalization()(relu)
  return bn

In [ ]:
def residual_block(x, downsample, filters, kernel_size = 3):
  y = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=(2 if downsample else 1), padding='same')(x)
  y = relu_bn(y)
  y = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(y)

  if downsample:
    x = layers.Conv2D(filters=filters, kernel_size=1, strides=2, padding='same')(x)

  out = layers.Add()([x, y])
  out = relu_bn(out)
  return out

In [ ]:
def resnet(input_shape, num_classes, apply_softmax=False):
  inputs = layers.Input(shape=input_shape)
  num_filters = 64
  num_blocks_list = [2, 5, 5, 2]
  t = layers.BatchNormalization()(inputs)
  t = layers.Conv2D(filters=num_filters, kernel_size=3, strides=1, padding='same')(t)
  t = relu_bn(t)

  for i in range(len(num_blocks_list)):
    num_blocks = num_blocks_list[i]
    for j in range(num_blocks):
      t = residual_block(t, (j == 0 and i != 0), filters=num_filters)

    num_filters *= 2

  t = layers.AveragePooling2D(4)(t)
  t = layers.Flatten()(t)
  outputs = layers.Dense(num_classes, activation='softmax' if apply_softmax else None)(t)
  
  model = tf.keras.models.Model(inputs, outputs)
  return model


In [ ]:
model = resnet(input_shape=(32, 32, 3), num_classes=10)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='resnet.png', show_dtype=True)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))